### Excercise 1

$ min_x (x_1 + 3x_2 + 3 x_3)\\
s.t. :\\ 
x_1 + 2x_2 >= 3\\
x_3 + 2x_2 >= 5\\
x_1 + x_3 = 4
$

In [20]:
import linopy
import xarray as xr
import warnings
import pandas as pd
from linopy import Model
warnings.filterwarnings("ignore")
m = Model()

In [21]:
"Variables"
x1 = m.add_variables(name = 'x1')
x2 = m.add_variables(name = 'x2')
x3 = m.add_variables(name = 'x3')

In [22]:
m.add_objective(x1+3*x2+3*x3)

In [23]:
m.add_constraints(x1 + 2*x2 >= 3)
m.add_constraints(x3 + x2 >= 5)
m.add_constraints(x1 + x3 == 4)

Constraint `con2`
-----------------
+1 x1 + 1 x3 = 4.0

In [24]:
m.solve()
print(linopy.available_solvers)

['highs', 'glpk']


### Printing solutions

In [25]:
print(m.objective, x1.solution,  x2.solution , x3.solution)

Objective:
----------
LinearExpression: +1 x1 + 3 x2 + 3 x3
Sense: min
Value: 15.333333333333334 <xarray.DataArray 'solution' ()>
array(0.33333333) <xarray.DataArray 'solution' ()>
array(1.33333333) <xarray.DataArray 'solution' ()>
array(3.66666667)


In [26]:
m.solution

<xarray.Dataset>
Dimensions:  ()
Data variables:
    x1       float64 0.3333
    x2       float64 1.333
    x3       float64 3.667

### Algebraic formulation of the objective function
$ min \sum_{i} X_{i}*Coef_{i} : i = [1,2,3] $

In [27]:
m2 = Model()

In [28]:
i = pd.Index(range(1,4), name = 'i')
coef = xr.DataArray([1 ,3 ,3], coords= [i]) 

In [29]:
X = m2.add_variables(lower = 0, coords = [i], name = "X")

### Using sum expression in linopy

In [30]:
(coef*X).sum()

LinearExpression
----------------
+1 X[1] + 3 X[2] + 3 X[3]

In [31]:
m2.add_objective((X*coef).sum())

# ¿ How can we write constraints using this approach?

In [32]:
"Selecting items by its position"
m2.add_constraints (X[1]+2*X[2]>=3)
"Selecting items by .loc Method"
m2.add_constraints (X.loc[3]+X.loc[2]>=5)
m2.add_constraints (X[1]+X[3]==4)

Constraint `con2`
-----------------
+1 X[1] + 1 X[3] = 4

In [33]:
m2.solve()
m2.objective

Objective:
----------
LinearExpression: +1 X[1] + 3 X[2] + 3 X[3]
Sense: min
Value: 15.333333333333334

In [34]:
m2.solution

<xarray.Dataset>
Dimensions:  (i: 3)
Coordinates:
  * i        (i) int64 1 2 3
Data variables:
    X        (i) float64 0.3333 1.333 3.667

### Load equations coeficients as DataSet

In [35]:
m3 = Model()

coef2 = pd.DataFrame({
    'eq1': [1,  2,  0],
    'eq2': [0,  1,  1],
    'eq3': [1,  0,  1]
})
# eq = pd.Index(["eq1", 'eq2', 'eq3'])

coef2 = xr.Dataset.from_dataframe(coef2.set_index(i))


In [36]:
X = m3.add_variables(lower = 0, coords = [i], name = "X")
m3.add_objective((coef*X).sum())
right = xr.DataArray([3 ,5 ,4], coords= [i])
m3.add_constraints((coef2['eq1']*X).sum()>=right[0])
m3.add_constraints((coef2['eq2']*X).sum()>=right[1])
m3.add_constraints((coef2['eq3']*X).sum()==right[2])


Constraint `con2`
-----------------
+1 X[1] + 0 X[2] + 1 X[3] = 4.0

In [37]:
m3.objective

Objective:
----------
LinearExpression: +1 X[1] + 3 X[2] + 3 X[3]
Sense: min
Value: None

In [38]:
m3.solve()
m3.solution, m3.objective

(<xarray.Dataset>
 Dimensions:  (i: 3)
 Coordinates:
   * i        (i) int64 1 2 3
 Data variables:
     X        (i) float64 0.3333 1.333 3.667,
 Objective:
 ----------
 LinearExpression: +1 X[1] + 3 X[2] + 3 X[3]
 Sense: min
 Value: 15.333333333333334)